In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

from tqdm import tqdm

import warnings
warnings.filterwarnings('once')

In [4]:
df=pd.read_csv('../data/04_registrados_cod_sta.csv', index_col=0)
df.head(2)

,año,registrados,mes_map,estacion_map,dia_semana_map,clima_map,vacaciones_map,laborable_cod,sensacion_termica_standar,humedad_standar,viento_standar
0,0,654,0,0,0,1,0,0,-0.680818,1.252343,-0.387833
1,0,670,0,0,0,1,1,1,-0.741507,0.480996,0.748899


In [5]:
#  separamos nuestro dataframe en X e y

X = df.drop(["registrados",'humedad_standar'], axis = 1)
y = df["registrados"]

In [6]:
# dividimos nuestros datos en train y test para poder evaluar la validez de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [31]:
param = {"max_depth": [6], 
        "max_features": [1,2,3],
        "min_samples_split": [40,50],
        "min_samples_leaf": [30,40]} 

In [32]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error")

In [33]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [6], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [30, 40],
                         'min_samples_split': [40, 50]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [34]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=6, max_features=3, min_samples_leaf=30,
                      min_samples_split=50)

In [35]:
# Calculamos sus métricas
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [36]:
# Con esta función calculamos las métricas y las convertimos en dataframe
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [37]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados
df_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
df_results

,MAE,MSE,RMSE,R2,set,modelo
0,640.678620,585053.163875,764.887681,0.728963,test,Random Forest
1,644.080022,673179.832021,820.475370,0.729981,train,Random Forest


.843161	test	Random Forest
1	403.188710	287281.152630	535.986150	0.884769

In [38]:
#df_todos_resultados.to_csv('06_randForest_registrados.csv')